# Area statistics for protected areas in mainland Norway

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb)

**Author**: Willeke A'Campo

**Description:** This notebook calculates area statistics for protected areas in mainland Norway. Area statistics are calculated for the datasetes **Verneområder** og **Foreslatte verneområder** from the Norwegian Environment Agency. 

The area statistics can be divided in three groups:

1. Area variables for the protected areas:
    - Area (terrestrial and marine)
    - Perimeter (terrestrial and marine)
    - Land area (terrestrial)
    - Perimeter land area (terrestrial)

2. Overlay statistics for the protected area and the following datasets:
    - AR50 - Bonitet
        - Area proportion of land cover quality classes 
    - Bioklimatiske soner
        - Area proportion of bioclimatic zone classes
    - Infrastrukturindeks: 
        - Mean value of infrastructure index per protected area 
        - Area proportion of infrastructure index classes
    - Høydelag
        - Area proportion of elevation classes 

3. Spatial indices for the protected areas:
    - Density of protected area per 10x10km (SSB grid)
    - Shape Index for each protected area (land + marine)
    - Shape index for each protected area (land)






In [7]:
import os 
from osgeo import gdal
from pathlib import Path

from shapely.geometry import box
import pandas as pd
import geopandas as gpd

# mapping libraries
# https://leafmap.org/faq/#how-to-use-a-specific-plotting-backend
import leafmap.foliumap as leafmap

In [12]:
project_path= Path.cwd().parents[0]
shell_path= os.path.join(project_path, "src", "shell")
python_path= os.path.join(project_path, "src", "python")
data_path = os.path.join(project_path, "data")

print(f"Gdal Version: {gdal.__version__} ")
print(f"Project Path: {project_path}")
print(f"Path to shell scripts: {shell_path}")
print(f"Path to python scripts: {python_path}")
print(f"Path to data: {data_path}")

Gdal Version: 3.6.3 
Project Path: /workspaces/pygdal-geo-engineer
Path to shell scripts: /workspaces/pygdal-geo-engineer/src/shell
Path to python scripts: /workspaces/pygdal-geo-engineer/src/python
Path to data: /workspaces/pygdal-geo-engineer/data


In [9]:
# import local scripts
import sys 
sys.path.append(python_path)

from ogr_utils import import_gpkg, print_layer_schema
from lookup import create_lookup_dict, lookup_value

## Import Data

Get the data and import into geopackages `vern_25833.gpkg` and `bevaring_25833.gpkg` using ogr2ogr.

```python
# change path to data folder
os.chdir(os.path.join(data_path, "raw"))
print(f"Download data to: {os.getcwd()}")
```

```bash
# ArcGIS REST: Naturvern (02.2024)
ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln naturvernomrade -nlt MULTIPOLYGON vern_25833.gpkg "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/0/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

# ArcGIS REST: Foreslåtte verneområder (02.2024)
ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln foreslatt_vern -nlt MULTIPOLYGON vern_25833.gpkg "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/4/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

# AR50 - 2022 
# Stored on NINA's server /GeoSpatialData

# Bioklima
# Received from V. Bakkestuen (NINA)

# SSB grid
curl -o Ruter_10KM_norge.zip "https://www.ssb.no/natur-og-miljo/_attachment/375082?_ts=1685c0e69b8"
unzip Ruter_10KM_norge.zip

# Fylkerområder
# Stored on NINA's server /GeoSpatialData

# Infrastruktur
# Received from V. Bakkestuen (NINA)
```

Downloaded datasets are stored in the `/data` folder:

**vern_25833.gpkg** contains the following layers:
| Layer | Dataset Name | Description | Year | Source |
|-------| ------------ | ----------- | ---- | ------ |
| naturvernomrade | Naturvernområder| Nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/0) |
| foreslatt_vern | Foreslåtte naturvernområder | Planned nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/1)| 

**bevaring_25833.gpkg** contains the following layers:
|Layer| Dataset Name | Description | Year | Source |
|-----| ------------ | ----------- | ---- | ------ |
| ar50_2022 | AR50 | Land cover classes | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard) |
| bioklima_soner_2017 | Bioklimatiske soner | Bioclimatic zones | 2017 | [Artsdatabanken](https://data.artsdatabanken.no/Natur_i_Norge/Natursystem/Beskrivelsessystem/Regional_naturvariasjon/Bioklimatisk_sone) |

**other files**

| Filename | Dataset Name | Description | Year | Source |
|----------| ------------ | ----------- | ---- | ------ |
| ssb_grid_5km.geojson | SSB rutenett (5x5 km)| Grid for Norway | 2024 | [SSB](https://kart.ssb.no/) |
| fylker_2024.geojson | Fylker, 2024 | Provincial Boundaries | 2024 | [SSB](https://www.ssb.no/en/kart/griddata) |
| Infra25m.tif | Infrastrukturindeks | Infrastructure index (25m)| 2024 | Internal NINA datasett |

In [29]:
# run shell scripts from /src/shell
os.chdir(shell_path)

input_file = "vern_25833.gpkg"
print(data_path)

! gdalinfo --version
! chmod +x gdal_gpkg-info.sh
! ./gdal_gpkg-info.sh {data_path} {input_file}

/workspaces/pygdal-geo-engineer/data
GDAL 3.6.3, released 2023/03/07


Input: vern_25833.gpkg
Process: logging info and writing metadata
Logfile: reproject.log
Metadata: metadata.txt


## Data Preparation

### Case Study | Hjerkinn, Dovrefjell

In this notebook we use Hjerkinn as example site to follow along the analysis steps on a map. Note that the analysis is performed for all protected areas in mainland Norway. 

https://ipyleaflet.readthedocs.io/en/latest/installation/index.html#using-pip 


In [ ]:
# Create a bounding box
dovreBB = box(minx=210000.00, miny=6920000.00, maxx=240000.00, maxy=6950000.00)
#trondheimBB = box(minx=260520.12, miny=7032142.5, maxx=278587.56, maxy=7045245.27)

crs = 'EPSG:25833'

gdf_BB = gpd.GeoDataFrame(geometry=[dovreBB])
gdf_BB['name'] = 'Dovre_BB'
gdf_BB.crs = crs
bounds = gdf_BB.bounds.to_numpy().tolist()[0]

# Display data on map using leafmap
init_location = [62.223207, 9.550195]  # Hjerkinn
zoom_start = 13  
min_zoom = 8  # keeps user from zooming out too far
basemap = leafmap.Map(
    location=init_location, 
    zoom=zoom_start, 
    min_zoom=min_zoom, 
    max_bounds=True
    )

# sat background
basemap.add_basemap("SATELLITE", opacity=0.7)

# add WMS verneområder
wms_url ="https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer"
wms_layer = "naturvern_klasser_omrade"
wms_name = "naturvern_klasser_omrade"
basemap.add_wms_layer(
    url=wms_url, 
    layers=wms_layer, 
    name=wms_name, 
    wms_format="image/png",
    )
# add bounding box - Hjerkinn
#basemap.add_gdf(gdf_BB, layer_name="Dovre_BB", fill_color="blue", fill_opacity=0.2, weight=2)

basemap

### AR50 - Bonitet

Translate the Bonitet classes using a lookup table.
 
&emsp; $Bonitet = (artype, artreslag, arskogbon, arjordbr, ardyrkning, arveget)$ 

In [ ]:
# ar50 layer
in_gpkg = os.path.join(data_path, "bevaring_25833.gpkg")
layer_name = "ar50_flate"
new_field_name = "ar50_bonitet"

# import gpkg into ogr object 
# add field name if it does not exist
ds, lyr = import_gpkg(in_gpkg, layer_name, new_field_name)
print_layer_schema(lyr)

In [ ]:
# lookup table 
lookup_csv = os.path.join(data_path, "AR50_bonitet_lookup.csv")
lookup_df = pd.read_csv(lookup_csv)

# rename cols to correspond with AR50 lyr
lookup_df.rename(
    columns={
        "ARTYPE kode": "artype",
        "ARTRESLAG kode": "artreslag",
        "ARSKOGBON kode": "arskogbon",
        "ARJORDBR kode": "arjordbr",
        "ARDYRKING kode": "ardyrking",
        "ARVEGET kode": "arveget",
        "Bonitet kode": "ar50_bonitet",
    },
    inplace=True,
)

# create lookup dict
keys = (
    "artype", 
    "artreslag", 
    "arskogbon", 
    "arjordbr", 
    "ardyrkning", 
    "arveget"
    )

value= "ar50_bonitet"

lookup_dict = create_lookup_dict(
    lookup_df,
    keys=keys,
    value=value
)

# lookup bonitet value
lookup_value(
    lyr, 
    lookup_dict, 
    keys=keys,
    value=value
)

# close OGR object
ds = None
print("Finished looking up Bonitet.")

In [ ]:
# merge marine areas (hav = 17) and terrestrial areas (all other codes)
# create new layer ar50_marine
# if ar50_bonitet = 17 --> copy + merge 

# create new layer ar50_terrestisk
# if ar50_bonitet != 17 --> copy + merge 

In [ ]:
# plot outline of terrestrial on the map 


### Protected Areas

Divide the protected areas into terrestrial and marine areas

Bonitet = 17 (Hav) are considered as marine areas, all other areas are considered as terrestrial areas.

### Infrastructure index

- Calculate and display the distribution of the infrastructure index for the whole of norway 
- Calculate and display the distribution of the infrastructure index per region
- Calculate and display the distribution of the infrastructure index per protected area
- Define the infrastructure index classes


### Topographic height

Classify the topographic height into 4 classes:
- 0-300m
- 301-600m
- 601-900m
- over 900m

## Methods
### 1. Geometry variables for protected areas

| Protected area | Total area (km2) | Perimeter (km) | Land area (km2) | Perimeter land area (km) |
|----------------|------------|----------------|-----------------|-------------------------|
| *NaturvernId* | *areal_km2* | *omkrets_km* | *landareal_km2* | *landomkrets_km*|
| area A         |    *        |                |                 |                         |

In [ ]:
# calc. landareal = area that overlaps with ar50_terrestrial
# calc. marineareal = area that overlaps with ar50_marine

***
$\mathbf{\text{Shape Index}}$<br>
***
The shape index is a measure of how compact the shape is compared to a circle with the same area. The shape index is calculated for the entire protected area and the land area. The shape index is a value between 0 and 1, where 0 is a perfect circle and 1 is a long and narrow shape.

- $P$ = perimeter
- $r$ = radius
- $A$ = area


**Shape Index:**&emsp;      $SI = \frac{P}{2\pi r}$

**Radius:**&emsp; $r = \sqrt{\frac{A}{\pi}}$

In [ ]:
# calc formindex

### 2. Area overlay statistics for the protected area and the following datasets:





**AR50 - Bonitet**

| Protected area | Fulldyrka og overflatedyrka jord (%) | Innmarksbeite (%) | Skog, høg og særs høg bonitet (%) | ... |
|----------------|--------------------------------------|-------------------|-----------------------------------|-----|
| *NaturvernId*  |*AR50_bon_1*|*AR50_bon_2*|*AR50_bon_3*|...|
| area A         |                                      |                   |                                   |     |


**Bioklimatiske soner**

| Protected area | Sone 1 (%) | Sone 2 (%) | Sone 3 (%) | ... |
|----------------|------------|------------|------------|-----|
| *NaturvernId*  |*bioklima_1*|*bioklima_2*|*bioklima_3*|...  |
| area A         |            |            |            |     |

**Infrastrukturindeks**

| Protected area | Mean value | Lav (%) | Middels (%) | Høy (%) | ... |
|----------------|------------|---------|-------------|---------|-----|
| area A         |            |         |             |         |     |


**Høydelag**

| Protected area | 0-300 m (%) | 301-600 m (%) | 601-900 m (%) | over 900 m (%) |
|----------------|-------------|---------------|---------------|----------------|
| area A         |             |               |               |                |


### 3. Regional Statistics


Administrative regions:
- Municipalities
- Counties
- Regions 
    - **Nord**: Finnmark, Troms, Nordland
    - **Midt**: Trøndelag, Møre og Romsdal
    - **Sør**: Agder, Vestfold, Telemark
    - **Øst**: Østfold, Akershus, Oslo, Innlandet, Buskerud
    - **Vest**: Vestland, Rogaland


| Protected area | Municipality | County | Region |
|----------------|---------|-------|--------|
| *NaturvernId*  |*kommune*|*fylke*|*region*|
| area A         |         |       |        |

*Regional datasett*:

| Region | Area | Protected Area | Land Protected Area | Marine Protected Area |
|--------|------|----------------|---------------------|-----------------------|
| *Region* | *areal* | *a_vern* | *a_landvern* | *a_marinvern* |
| Nord ||||||


###  4. Grid Statistics (10x10 km2)

**Rutenettstatistikk:**

| SSB grid cell | area | Density (land + marine) | Density (land) | 
| --------------|------|-------------------------|----------------|
| *grid_ID* | *grid_areal* | *tetthet_tot_vern* | *tetthet_landvern* | 
| cell 1| 100 km2 | ||





***
$\mathbf{\text{Density}}$<br>
***
The density of protected area per 10x10km (SSB grid) is calculated by dividing the area of the protected area by 100 km^2.

**Density:**&emsp;      $Density = \frac{A}{100 km^2}$



In [ ]:
# code for density calculation here
# check land cover fraction notebook
